In [65]:
import pandas as pd

attendance = pd.read_csv('data/attendance.csv')
entity_schedule = pd.read_csv('data/entity_schedule.csv')
link_atrraction_park = pd.read_csv('data/link_attraction_park.csv')
waiting_times = pd.read_csv('data/waiting_times.csv')
weather_data = pd.read_csv('data/weather_data.csv')

In [ ]:
from tqdm import tqdm

# Filtering for only Port Aventura rides
filtered_attractions = link_atrraction_park[link_atrraction_park["ATTRACTION;PARK"].str.contains("PortAventura World")]
ride_names = filtered_attractions["ATTRACTION;PARK"].str.split(";", expand=True)[0]
waiting_times = waiting_times[waiting_times["ENTITY_DESCRIPTION_SHORT"].isin(ride_names)]

# Excluding Tivoli Gardens park and Tivoli Gardens rides
entity_schedule = entity_schedule[entity_schedule["ENTITY_DESCRIPTION_SHORT"] != "Tivoli Gardens"]
park_closures = entity_schedule[entity_schedule["ENTITY_TYPE"] == "PARK"]

entity_schedule_rides = entity_schedule[entity_schedule["ENTITY_DESCRIPTION_SHORT"].isin(ride_names)]
ride_closures = entity_schedule[entity_schedule["ENTITY_TYPE"] == "ATTR"]


# Converting to datetime
entity_schedule["DEB_TIME"] = pd.to_datetime(entity_schedule["DEB_TIME"])
entity_schedule["FIN_TIME"] = pd.to_datetime(entity_schedule["FIN_TIME"])

waiting_times["DEB_TIME"] = pd.to_datetime(waiting_times["DEB_TIME"])
waiting_times["FIN_TIME"] = pd.to_datetime(waiting_times["FIN_TIME"])


for index, row in tqdm(park_closures.iterrows(), total=park_closures.shape[0]):
    deb_time = pd.to_datetime(row['DEB_TIME'])
    fin_time = pd.to_datetime(row['FIN_TIME'])
    waiting_times = waiting_times[~((waiting_times['DEB_TIME'] >= deb_time) & (waiting_times['DEB_TIME'] <= fin_time))]

for index, row in tqdm(ride_closures.iterrows(), total=ride_closures.shape[0]):
    deb_time = pd.to_datetime(row['DEB_TIME'])
    fin_time = pd.to_datetime(row['FIN_TIME'])
    waiting_times = waiting_times[~((waiting_times['DEB_TIME'] >= deb_time) & (waiting_times['DEB_TIME'] <= fin_time) & (waiting_times['ENTITY_DESCRIPTION_SHORT'] == row['ENTITY_DESCRIPTION_SHORT']))]

  5%|▍         | 1697/34579 [04:12<1:18:49,  6.95it/s]

In [ ]:
waiting_times["DEB_TIME"] = pd.to_datetime(waiting_times["DEB_TIME"])
waiting_times["FIN_TIME"] = pd.to_datetime(waiting_times["FIN_TIME"])
test = waiting_times.iloc[0:2]
test = test[~((test['DEB_TIME'] >= pd.to_datetime("2018-01-01 20:00:00")) & (test['DEB_TIME'] <= pd.to_datetime("2018-01-01 23:00:00")))]

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT
1,2018-01-01,2018-01-01 19:30:00,19,2018-01-01 19:45:00,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0


In [29]:
waiting_times

,WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT
0,2018-01-01,2018-01-01 21:00:00.000,21,2018-01-01 21:15:00.000,Roller Coaster,0,2.0,0.0,0.000,0.00,0,0,0,2.0
1,2018-01-01,2018-01-01 19:30:00.000,19,2018-01-01 19:45:00.000,Bumper Cars,5,18.0,148.0,254.749,254.75,15,15,0,18.0
2,2018-01-01,2018-01-01 22:30:00.000,22,2018-01-01 22:45:00.000,Rapids Ride,0,1.0,0.0,0.000,0.00,0,0,0,2.0
3,2018-01-01,2018-01-01 12:45:00.000,12,2018-01-01 13:00:00.000,Crazy Dance,5,1.0,46.0,250.001,250.00,15,15,0,1.0
5,2018-01-01,2018-01-01 18:15:00.000,18,2018-01-01 18:30:00.000,Free Fall,50,3.0,0.0,0.000,0.00,0,0,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509319,2022-08-18,2022-08-18 18:45:00.000,18,2022-08-18 19:00:00.000,Himalaya Ride,0,0.0,0.0,0.000,0.00,0,0,0,1.0
3509320,2022-08-18,2022-08-18 10:15:00.000,10,2022-08-18 10:30:00.000,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0
3509321,2022-08-18,2022-08-18 09:15:00.000,9,2022-08-18 09:30:00.000,Crazy Dance,0,0.0,0.0,0.000,0.00,0,0,0,1.0
3509322,2022-08-18,2022-08-18 20:30:00.000,20,2022-08-18 20:45:00.000,Giga Coaster,0,0.0,0.0,0.000,0.00,0,0,0,24.0


In [ ]:
# Processing weather data
relevant_weather_data = weather_data[['dt_iso', 'temp', 'humidity', 'wind_speed', 'clouds_all', 'rain_1h', 'snow_1h']]
relevant_weather_data['dt_iso'] = relevant_weather_data['dt_iso'].str.split('+').str[0]
relevant_weather_data['date'] = pd.to_datetime(relevant_weather_data['dt_iso'].str.split(' ').str[0])
relevant_weather_data['time'] = relevant_weather_data['dt_iso'].str.split(' ').str[1]
relevant_weather_data['hour'] = relevant_weather_data['time'].str.split(':').str[0].astype(int) + 1
relevant_weather_data = relevant_weather_data.drop(columns=['dt_iso', 'time'])
relevant_weather_data.fillna(0, inplace=True)

# One hot encoding times
waiting_times['DEB_TIME'] = waiting_times['DEB_TIME'].astype(str)
waiting_times['date'] = pd.to_datetime(waiting_times['DEB_TIME'].str.split(' ').str[0])
waiting_times['time'] = waiting_times['DEB_TIME'].str.split(' ').str[1]

# Encode the hour and quarter linearly in the time column
waiting_times['hour'] = waiting_times['time'].str.split(':').str[0].astype(int)
waiting_times['minute'] = waiting_times['time'].str.split(':').str[1].astype(int)
waiting_times['time_encoded'] = waiting_times['hour'] * 4 + waiting_times['minute'] / 15

# Merge the weather data
waiting_times = pd.merge(waiting_times, relevant_weather_data, on=['date', 'hour'], how='left')

In [4]:
import numpy as np

waiting_times['day_of_week'] = waiting_times['date'].dt.dayofweek
waiting_times['day_of_week_sin'] = np.sin(2 * np.pi * waiting_times['day_of_week'] / 7)
waiting_times['day_of_week_cos'] = np.cos(2 * np.pi * waiting_times['day_of_week'] / 7)

waiting_times['year'] = waiting_times['date'].dt.year
waiting_times['month'] = waiting_times['date'].dt.month
waiting_times['day'] = waiting_times['date'].dt.day
waiting_times = pd.get_dummies(waiting_times, columns=['year', 'month', 'day'])

waiting_times.drop(columns=['time', 'hour', 'minute', 'day_of_week'], inplace=True)

In [5]:
# One hot encoding ride names
ride_names = waiting_times['ENTITY_DESCRIPTION_SHORT'].unique()
ride_names_encoded = pd.get_dummies(waiting_times['ENTITY_DESCRIPTION_SHORT'])
waiting_times = pd.concat([waiting_times, ride_names_encoded], axis=1)

In [6]:
# Dropping any rows with downtime for training
waiting_times = waiting_times[waiting_times["DOWNTIME"] == 0]

# Dropping columns that are not useful for training
waiting_times.drop(columns=["WORK_DATE", "FIN_TIME", "NB_UNITS", "GUEST_CARRIED", "NB_MAX_UNIT",
                            "ADJUST_CAPACITY", "OPEN_TIME", "UP_TIME", "CAPACITY", "DOWNTIME",
                            "ENTITY_DESCRIPTION_SHORT"], inplace=True)

In [7]:
# Adding attendance as a feature
attendance = attendance[attendance["FACILITY_NAME"] == "PortAventura Park"].rename(columns={"USAGE_DATE": "date"})
waiting_times = pd.merge(waiting_times, attendance, on="date", how="left")

In [ ]:
# Taking into account entity schedule
entity_schedule = entity_schedule[entity_schedule["ENTITY_DESCRIPTION_SHORT"] != "Tivoli Gardens"]
entity_schedule = entity_schedule[entity_schedule["ENTITY_DESCRIPTION_SHORT"].isin(ride_names)]
park_closures = entity_schedule[entity_schedule["ENTITY_TYPE"] == "PARK"]
ride_closures = entity_schedule[entity_schedule["ENTITY_TYPE"] == "ATTRACTION"]

for index, row in park_closures.iterrows():
    deb_time = pd.to_datetime(row['DEB_TIME'])
    fin_time = pd.to_datetime(row['FIN_TIME'])
    waiting_times = waiting_times[~((waiting_times['DEB_TIME'] >= deb_time) & (waiting_times['DEB_TIME'] <= fin_time))]

for index, row in ride_closures.iterrows():
    deb_time = pd.to_datetime(row['DEB_TIME'])
    fin_time = pd.to_datetime(row['FIN_TIME'])
    waiting_times = waiting_times[~((waiting_times['DEB_TIME'] >= deb_time) & (waiting_times['DEB_TIME'] <= fin_time) & (waiting_times['ENTITY_DESCRIPTION_SHORT'] == row['ENTITY_DESCRIPTION_SHORT']))]

In [9]:
waiting_times.columns

Index(['DEB_TIME_HOUR', 'WAIT_TIME_MAX', 'date', 'time_encoded', 'temp',
       'humidity', 'wind_speed', 'clouds_all', 'rain_1h', 'snow_1h',
       'day_of_week_sin', 'day_of_week_cos', 'year_2018', 'year_2019',
       'year_2020', 'year_2021', 'year_2022', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12', 'day_1', 'day_2', 'day_3', 'day_4',
       'day_5', 'day_6', 'day_7', 'day_8', 'day_9', 'day_10', 'day_11',
       'day_12', 'day_13', 'day_14', 'day_15', 'day_16', 'day_17', 'day_18',
       'day_19', 'day_20', 'day_21', 'day_22', 'day_23', 'day_24', 'day_25',
       'day_26', 'day_27', 'day_28', 'day_29', 'day_30', 'day_31',
       'Bumper Cars', 'Bungee Jump', 'Circus Train', 'Crazy Dance',
       'Dizzy Dropper', 'Drop Tower', 'Flying Coaster', 'Free Fall',
       'Giant Wheel', 'Giga Coaster', 'Go-Karts', 'Haunted House',
       'Himalaya Ride', 'Inverted Coaster', 'Kiddie Coaster

In [12]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import KFold

# Define the number of splits for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store the results
train_rmse_list = []
val_rmse_list = []

cutoff_date = waiting_times['date'].max() - pd.DateOffset(months=3)
train_set = waiting_times[waiting_times['date'] < cutoff_date]
val_set = waiting_times[waiting_times['date'] >= cutoff_date]
X_train = train_set.drop(columns=['WAIT_TIME_MAX', 'date'])
y_train = train_set['WAIT_TIME_MAX']
X_val = val_set.drop(columns=['WAIT_TIME_MAX', 'date'])
y_val = val_set['WAIT_TIME_MAX']

# Train the final model on the entire training data
model = XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)

# Predict on the final validation set
y_pred_final = model.predict(X_val)

# Evaluate the final model
final_rmse = root_mean_squared_error(y_val, y_pred_final)  
print(f'Final Validation RMSE: {final_rmse}')

[0]	validation_0-rmse:26.84190
[100]	validation_0-rmse:18.64033
[200]	validation_0-rmse:17.65496
[300]	validation_0-rmse:16.96661
[400]	validation_0-rmse:16.64932
[500]	validation_0-rmse:16.50481
[600]	validation_0-rmse:16.51935
[700]	validation_0-rmse:16.55748
[800]	validation_0-rmse:16.46327


KeyboardInterrupt: 